In [1]:
#https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html?highlight=autoreload
%load_ext autoreload
%autoreload 2

import json
import time
import os
from datetime import datetime
import pandas as pd

from OpenWeatherMap import OpenWeatherMap
from Kafka import KafkaConfluentWriter

In [2]:
openWeatherMap = OpenWeatherMap()
kafkaConfluentWriter = KafkaConfluentWriter('weather.forecast')

def format_date(dt: str) -> str:
    return datetime.fromtimestamp(float(dt)).strftime('%d.%m.%Y %H:%M')


# load 'locations.json' into a json-object and return it
def load_locations() -> json:
    with open('locations.json', mode='r') as file:
        return json.load(file)

def get_transformed_forecast(cities: json) -> {}:
    forecasts = []

    for key in cities:
        city = cities[key]
        forecast = openWeatherMap.get_forecast(city)

        value = {
            'city': forecast['city']['name'],
            'weather': []
        }

        preds = []
        for pred in forecast['list']:
            preds.append({
                'dt': pred['dt_txt'],
                'temp': pred['main']['temp']
            })
        value['weather'] = preds

        forecasts.append(value)
    cities = {
        'cities': forecasts
    }
    return cities

# for each location, query openWeatherMap for the 5-day forecast and 
# store the the returned values in Kafka
def collect_forecast_data() -> None:
    try:
        print("Starting collection ...")

        cities = load_locations()

        while True:
            forecasts = get_transformed_forecast(cities)
            dt = format_date(datetime.timestamp(datetime.now()))

            print(f"Weather data collected at {dt}!")

            kafkaConfluentWriter.produce(dt, forecasts)

            time.sleep(60)

    except KeyboardInterrupt:
        print("... collection stopped!")

In [3]:
collect_forecast_data()

Starting collection ...
Weather data collected at 24.08.2022 21:43!
Message delivered to weather.forecast [0]
Weather data collected at 24.08.2022 21:44!
Message delivered to weather.forecast [0]
Weather data collected at 24.08.2022 21:45!
Message delivered to weather.forecast [0]
Weather data collected at 24.08.2022 21:46!
Message delivered to weather.forecast [0]
Weather data collected at 24.08.2022 21:47!
Message delivered to weather.forecast [0]
Weather data collected at 24.08.2022 21:48!
Message delivered to weather.forecast [0]
Weather data collected at 24.08.2022 21:49!
Message delivered to weather.forecast [0]
Weather data collected at 24.08.2022 21:50!
Message delivered to weather.forecast [0]
... collection stopped!
